In [11]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options # 크롬의 옵션 설정 모듈
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
import time

In [12]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
chromeOptions.add_experimental_option("prefs", prefs)
#chromeOptions.add_argument('headless')
chromeOptions.add_argument("window-size=1200x600")     # 브라우저 크기
chromeOptions.add_argument("disable-gpu")                # gpu 사용 안함
chromeOptions.add_argument("user-agent=Mozilla/5.0")  # user-agent 덮어씌우기

#chromeOptions.add_experimental_option("prefs", prefs)
#driver = webdriver.Chrome(chrome_options=chromeOptions)

In [13]:
#url 크롤링하는 함수 정의

def crawl_link(html_text, link_name):
    #req = requests.get('https://www.computer.org'+next_url)
    soup = bs(html_text, 'html.parser')
    next_r_url = soup.select(link_name)
    
    parsed = [] 
    for i in range(0,len(next_r_url)):
        parsed.append(next_r_url[i].get("href")) #url 추출해서 list에 append
    
    return parsed #list 반환

In [14]:
browser = webdriver.Chrome("C:\kcg\ChromeDriver\chromedriver.exe",chrome_options=chromeOptions)
browser.implicitly_wait(1)
browser.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
browser.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")

c:\users\곽주호\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [15]:
url_main = "https://www.computer.org"
browser.get(url_main+"/csdl/journals")
print("Access to the journals Page")
#Wait Until the page shown

element = WebDriverWait(browser, 30).until(
EC.presence_of_element_located((By.CLASS_NAME,'ml-sm'))
)
    

html = browser.page_source
journals = crawl_link(html, 'a.btn.btn-default.ml-sm')
print(journals)

Access to the journals Page
['/csdl/journal/cq', '/csdl/journal/ca', '/csdl/journal/dc', '/csdl/journal/lc', '/csdl/journal/ta', '/csdl/journal/bd', '/csdl/journal/cc', '/csdl/journal/ci', '/csdl/journal/tc', '/csdl/journal/tq', '/csdl/journal/ec', '/csdl/journal/th', '/csdl/journal/tk', '/csdl/journal/lt', '/csdl/journal/tm', '/csdl/journal/mc', '/csdl/journal/tn', '/csdl/journal/td', '/csdl/journal/tp', '/csdl/journal/sc', '/csdl/journal/ts', '/csdl/journal/su', '/csdl/journal/si', '/csdl/journal/tg', '/csdl/journal/tb', '/csdl/journal/nt']


In [10]:
len(journals)

26

In [106]:
article_links = []
def crawlArticleLinks(journals) :
    for i in range(0, len(journals)) :
        url_journal = url_main+journals[i]
        browser.get(url_journal)
        element = WebDriverWait(browser, 60).until(
        EC.presence_of_element_located((By.CLASS_NAME,'nav-link'))
        )
        
        btn = browser.find_element_by_css_selector("a[aria-controls='pastIssuesMenu']")
        btn.click()
        element = WebDriverWait(browser, 60).until(
        EC.presence_of_element_located((By.CLASS_NAME,'csdl-cover-image'))
        )
                     
        #year, volume 크롤링
        article_links = []
        year = 2018
        y_v = {}
        first_article = "none"
        #num = 3

        try:
            btn = browser.find_element_by_link_text("2010s")
            btn.click()

            print("start 2010s")
            while True :
                btn = browser.find_element_by_link_text( str(year) )
                btn.click()

                html = browser.page_source
                soup = bs(html, 'html.parser')
                while ( str(year) != soup.select_one("#pastIssuesMenu > div > div > div:nth-of-type(1) > div > a > h2").string ) :
                    print("no change!! Wait javascript")
                    btn.click()
                    html = browser.page_source
                    soup = bs(html, 'html.parser')
                btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
                print(year," : ", len(btns))
                y_v[year] = len(btns)
                year -= 1
        except Exception as e:
            #print(e)
            print("finish 2010s")

        try:
            btn = browser.find_element_by_link_text("2000s")
            btn.click()

            print("start 2000s")
            while True :
                btn = browser.find_element_by_link_text( str(year) )
                btn.click()

                html = browser.page_source
                soup = bs(html, 'html.parser')
                while ( str(year) != soup.select_one("#pastIssuesMenu > div > div > div:nth-of-type(1) > div > a > h2").string ) :
                    print("no change!! Wait javascript")
                    btn.click()
                    html = browser.page_source
                    soup = bs(html, 'html.parser')
                btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
                print(year," : ", len(btns))
                y_v[year] = len(btns)
                year -= 1
        except Exception:
            print("finish 2000s")
                     
        #논문 주소 크롤링
        for year in y_v :
            for vol in range(1, y_v[year]+1) :
                if vol/10 != 0 :
                    v = "/0" + str(vol)
                else :
                    v = "/0" + str(vol)
                browser.get(url_journal + '/' + str(year) + v)
                element = WebDriverWait(browser, 60).until(
                EC.presence_of_element_located((By.CLASS_NAME,'content-section'))
                )

                links = browser.find_elements_by_css_selector("body > csdl-app > div > csdl-periodical-details > div.mb > csdl-periodical-issue > div > div > div.col-md-8.col-xs-12 > div > csdl-article-list-item > div > div > a.article-title")
                print(str(year) + '_' + str(vol) + ' : ' + str(len(links)) )
                for a in links :
                    print(a.get_attribute('href'))
                    article_links.append(a.get_attribute('href'))
                    
        print(i, " of ", len(journals), " complete...")

In [107]:
article_links = crawlArticleLinks(journals)

start 2010s
2018  :  10
no change!! Wait javascript
finish 2010s
finish 2000s
2018_1 : 0
2018_2 : 0
2018_3 : 0
2018_4 : 0
2018_5 : 0
2018_6 : 0
2018_7 : 0
2018_8 : 0
2018_9 : 0
2018_10 : 0
start 2010s
2018  :  2
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
2017  :  2
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
2016  :  2
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!

https://www.computer.org/csdl/journal/ca/2018/02/08425743/13xI8B2zWrH
https://www.computer.org/csdl/journal/ca/2018/02/08444676/13xI8A8WyX3
https://www.computer.org/csdl/journal/ca/2018/02/08419335/14ZDKJtqPMQ
https://www.computer.org/csdl/journal/ca/2018/02/08419212/14ZDLkaecbS
https://www.computer.org/csdl/journal/ca/2018/02/08423062/14ZDLbCyHtu
https://www.computer.org/csdl/journal/ca/2018/02/08437148/14ZDLZxpcEU
https://www.computer.org/csdl/journal/ca/2018/02/08440730/14ZDLF7qexa
https://www.computer.org/csdl/journal/ca/2018/02/08444757/14ZDLvYzhMk
https://www.computer.org/csdl/journal/ca/2018/02/08481514/14ZDMp3jIys
https://www.computer.org/csdl/journal/ca/2018/02/08482332/14ZDKZbCacg
2017_1 : 22
https://www.computer.org/csdl/journal/ca/2017/01/07366753/13rRUwh80IS
https://www.computer.org/csdl/journal/ca/2017/01/07425206/13rRUwh80Js
https://www.computer.org/csdl/journal/ca/2017/01/07544479/13rRUyoPSR0
https://www.computer.org/csdl/journal/ca/2017/01/07530858/13rRUB6SpUh
https://

https://www.computer.org/csdl/journal/ca/2015/01/06842649/13rRUB7a193
https://www.computer.org/csdl/journal/ca/2015/01/06803888/13rRUxlgxOR
https://www.computer.org/csdl/journal/ca/2015/01/06704732/13rRUwh80wj
https://www.computer.org/csdl/journal/ca/2015/01/07128794/13rRUwgQpki
https://www.computer.org/csdl/journal/ca/2015/01/07128798/13rRUxjyX9c
2015_2 : 24
https://www.computer.org/csdl/journal/ca/2015/02/07366624/13rRUwbs1Uq
https://www.computer.org/csdl/journal/ca/2015/02/07366620/13rRUxbCbkE
https://www.computer.org/csdl/journal/ca/2015/02/06936844/13rRUB7a138
https://www.computer.org/csdl/journal/ca/2015/02/06907961/13rRUB6SpRt
https://www.computer.org/csdl/journal/ca/2015/02/07243305/13rRUxcsYQT
https://www.computer.org/csdl/journal/ca/2015/02/06949088/13rRUzpzeEK
https://www.computer.org/csdl/journal/ca/2015/02/06860268/13rRUwciPeI
https://www.computer.org/csdl/journal/ca/2015/02/06912980/13rRUB7a137
https://www.computer.org/csdl/journal/ca/2015/02/06928476/13rRUzpzeDR
https://

2011_2 : 7
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020029/13rRUwfqpIn
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020033/13rRUyogGC9
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020037/13rRUwhpBG6
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020041/13rRUy08Myh
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020045/13rRUx0ge7U
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020049/13rRUxZRbqN
https://www.computer.org/csdl/journal/ca/2011/02/lca2011020053/13rRUxcsYNW
2010_1 : 9
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010001/13rRUxCitAy
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010005/13rRUwhpBPF
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010009/13rRUxCitAz
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010013/13rRUwciPgy
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010017/13rRUy3gn32
https://www.computer.org/csdl/journal/ca/2010/01/lca2010010021/13rRUzpzeCY
htt

start 2010s
2018  :  2
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
2017  :  1
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
2016  :  1
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
2015  :  1
finish 2010s
finish 2000s
2018_1 : 15
https://www.computer.org/csdl/journal/dc/2018/01/08399135/13rRUyft7yy
https://www.computer.org/csdl/journal/dc/2018/01/08302839/13rRUwh80OK
https://ww

...

2016_4 : 7
https://www.computer.org/csdl/journal/bd/2016/04/07582489/13rRUwhpBPB
https://www.computer.org/csdl/journal/bd/2016/04/07726017/13rRUxCitBh
https://www.computer.org/csdl/journal/bd/2016/04/07547363/13rRUxCitBg
https://www.computer.org/csdl/journal/bd/2016/04/07723926/13rRUx0xPuX
https://www.computer.org/csdl/journal/bd/2016/04/07587384/13rRUxNmPFQ
https://www.computer.org/csdl/journal/bd/2016/04/07592411/13rRUx0Pquw
https://www.computer.org/csdl/journal/bd/2016/04/07563854/13rRUxjyX99
2015_1 : 4
https://www.computer.org/csdl/journal/bd/2015/01/07265157/13rRUwwslvh
https://www.computer.org/csdl/journal/bd/2015/01/07265156/13rRUygT7c8
https://www.computer.org/csdl/journal/bd/2015/01/07230259/13rRUwvT9j1
https://www.computer.org/csdl/journal/bd/2015/01/07153539/13rRUxBJhxj
2015_2 : 2
https://www.computer.org/csdl/journal/bd/2015/02/07239545/13rRUxAStUe
https://www.computer.org/csdl/journal/bd/2015/02/07321000/13rRUxASu2L
2015_3 : 4
https://www.computer.org/csdl/journal/bd/2015/

2017_1 : 14
https://www.computer.org/csdl/journal/cc/2017/01/07012089/13rRUyfKIK3
https://www.computer.org/csdl/journal/cc/2017/01/07872549/13rRUy3xYcG
https://www.computer.org/csdl/journal/cc/2017/01/07012089/13rRUyfKIK3
https://www.computer.org/csdl/journal/cc/2017/01/07018917/13rRUB7a194
https://www.computer.org/csdl/journal/cc/2017/01/07027809/13rRUNvPLbK
https://www.computer.org/csdl/journal/cc/2017/01/07018931/13rRUxNmPKG
https://www.computer.org/csdl/journal/cc/2017/01/06963393/13rRUygT7cc
https://www.computer.org/csdl/journal/cc/2017/01/06987311/13rRUIJcWnM
https://www.computer.org/csdl/journal/cc/2017/01/06995996/13rRUxC0SxW
https://www.computer.org/csdl/journal/cc/2017/01/07039233/13rRUxlgxOS
https://www.computer.org/csdl/journal/cc/2017/01/07006776/13rRUEgarpB
https://www.computer.org/csdl/journal/cc/2017/01/07029041/13rRUwjoNz8
https://www.computer.org/csdl/journal/cc/2017/01/07872548/13rRUxOvebQ
https://www.computer.org/csdl/journal/cc/2017/01/07872546/13rRUNvPLbL
2017_2 :

2015_2 : 13
https://www.computer.org/csdl/journal/cc/2015/02/07118806/13rRUB7a1hQ
https://www.computer.org/csdl/journal/cc/2015/02/07118807/13rRUygBw1I
https://www.computer.org/csdl/journal/cc/2015/02/07015569/13rRUy0ZzUv
https://www.computer.org/csdl/journal/cc/2015/02/06888495/13rRUx0xPCm
https://www.computer.org/csdl/journal/cc/2015/02/06910293/13rRUB7a13b
https://www.computer.org/csdl/journal/cc/2015/02/06881664/13rRUwkfB1t
https://www.computer.org/csdl/journal/cc/2015/02/06881644/13rRUx0PquC
https://www.computer.org/csdl/journal/cc/2015/02/06898826/13rRUxcsYNY
https://www.computer.org/csdl/journal/cc/2015/02/07005434/13rRUxBa5zn
https://www.computer.org/csdl/journal/cc/2015/02/06942233/13rRUwh80Jv
https://www.computer.org/csdl/journal/cc/2015/02/06868272/13rRUxjQyjf
https://www.computer.org/csdl/journal/cc/2015/02/07089268/13rRUwx1xJr
https://www.computer.org/csdl/journal/cc/2015/02/06905754/13rRUB7a13a
2015_3 : 13
https://www.computer.org/csdl/journal/cc/2015/03/07235008/13rRUygT

2009  :  12
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no change!! Wait javascript
no chang

TimeoutException: Message: 


In [108]:
len(article_links)

0

In [ ]:
from datetime import datetime

#파일명을 현재 시간으로
now = str(datetime.now())[0:19]
filename = ("./output/" + "%s" + "_output.txt") % now
filename = filename.replace(':','-')

start = time.time()

#크롤링 하는 함수
def crawl_abs(link) :
    browser.get(link)
    element = WebDriverWait(browser, 60).until(
    EC.presence_of_element_located((By.CLASS_NAME,'article-title'))
    )
    html = browser.page_source
    soup = bs(html, 'html.parser')
    
    journal = {}
    journal['title'] = soup.select_one("div.article-max-width > csdl-periodical-article-metadata > div > div.article-title").text
    journal['keyword'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > div.mt > span").text[9:]
    journal['abstract'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-article-full-text > div.article-content.mt-lg").text[8:]
    journal['date'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > span:nth-of-type(1)").text
    
    with open(filename, 'a') as f:
        #각 요소를 tap으로 구분해서 쓰기
        o_data = '%s\t%s\t%s\t%s\n' % (journal['date'], journal['title'], journal['keyword'], journal['abstract'])
        #cp949 인코딩 문제 우선 임시적으로 이렇게 해결
        o_data = o_data.replace('\u2014','')
        o_data = o_data.replace('\u2013','')
        print(o_data)
        f.write(o_data)

for link in article_links :
    crawl_abs(link)
        
close = time.time()
print(close-start)

In [ ]:
#pandas를 이용하여 output파일을 읽어오기
data = pd.read_csv(filename, sep="\t", header=None)

#column 이름 설정
data.columns = ['Date','Title', 'Keyword', 'Contents']
data

### test

In [16]:
url_journal = url_main+journals[1]
browser.get(url_journal)
element = WebDriverWait(browser, 60).until(
EC.presence_of_element_located((By.CLASS_NAME,'nav-link'))
)
btn = browser.find_element_by_css_selector("a[aria-controls='pastIssuesMenu']")
btn.click()
element = WebDriverWait(browser, 60).until(
EC.presence_of_element_located((By.CLASS_NAME,'csdl-cover-image'))
)

In [17]:
#year, volume 크롤링
article_links = []
year = 2018
y_v = {}
first_article = "none"
#num = 3

try:
    btn = browser.find_element_by_link_text("2010s")
    btn.click()
    
    print("start 2010s")
    while True :
        btn = browser.find_element_by_link_text( str(year) )
        btn.click()
            
        html = browser.page_source
        soup = bs(html, 'html.parser')
        while ( str(year) != soup.select_one("#pastIssuesMenu > div > div > div:nth-of-type(1) > div > a > h2").string ) :
            #print("no change!! Wait javascript")
            btn.click()
            html = browser.page_source
            soup = bs(html, 'html.parser')
        btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
        print(year," : ", len(btns))
        y_v[year] = len(btns)
        year -= 1
except Exception as e:
    #print(e)
    print("finish 2010s")

try:
    btn = browser.find_element_by_link_text("2000s")
    btn.click()
    
    print("start 2000s")
    while True :
        btn = browser.find_element_by_link_text( str(year) )
        btn.click()
            
        html = browser.page_source
        soup = bs(html, 'html.parser')
        while ( str(year) != soup.select_one("#pastIssuesMenu > div > div > div:nth-of-type(1) > div > a > h2").string ) :
            #print("no change!! Wait javascript")
            btn.click()
            html = browser.page_source
            soup = bs(html, 'html.parser')
        btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
        print(year," : ", len(btns))
        y_v[year] = len(btns)
        year -= 1
except Exception:
    print("finish 2000s")

start 2010s
2018  :  2
2017  :  2
2016  :  2
2015  :  2
2014  :  2
2013  :  2
2012  :  2
2011  :  2
2010  :  2
finish 2010s
start 2000s
2009  :  2
2008  :  2
2007  :  2
2006  :  2
2005  :  1
2004  :  1
2003  :  1
2002  :  1
finish 2000s


In [18]:
#논문 주소 크롤링
for year in y_v :
    for vol in range(1, y_v[year]+1) :
        if vol/10 != 0 :
            v = "/0" + str(vol)
        else :
            v = "/0" + str(vol)
        browser.get(url_journal + '/' + str(year) + v)
        element = WebDriverWait(browser, 60).until(
        EC.presence_of_element_located((By.CLASS_NAME,'article-title'))
        )
        
        links = browser.find_elements_by_css_selector("body > csdl-app > div > csdl-periodical-details > div.mb > csdl-periodical-issue > div > div > div.col-md-8.col-xs-12 > div > csdl-article-list-item > div > div > a.article-title")
        print(str(year) + '_' + str(vol) + ' : ' + str(len(links)) )
        for a in links :
            print(a.get_attribute('href'))
            article_links.append(a.get_attribute('href'))

2018_1 : 26
https://www.computer.org/csdl/journal/ca/2018/01/08107529/13rRUwhpBSc
https://www.computer.org/csdl/journal/ca/2018/01/07911285/13rRUwhpBHY
https://www.computer.org/csdl/journal/ca/2018/01/07917248/13rRUIJcWfA
https://www.computer.org/csdl/journal/ca/2018/01/07919232/13rRUy0HYTG
https://www.computer.org/csdl/journal/ca/2018/01/07932455/13rRUxCitLq
https://www.computer.org/csdl/journal/ca/2018/01/07938603/13rRUB7a1hO
https://www.computer.org/csdl/journal/ca/2018/01/07946089/13rRUx0PquB
https://www.computer.org/csdl/journal/ca/2018/01/07953484/13rRUypGGcF
https://www.computer.org/csdl/journal/ca/2018/01/07956156/13rRUy0HYTH
https://www.computer.org/csdl/journal/ca/2018/01/07967722/13rRUzp02hT
https://www.computer.org/csdl/journal/ca/2018/01/08031080/13rRUzp02hU
https://www.computer.org/csdl/journal/ca/2018/01/08031326/13rRUyp7u0W
https://www.computer.org/csdl/journal/ca/2018/01/08039166/13rRUwvT9lj
https://www.computer.org/csdl/journal/ca/2018/01/08049259/13rRUwvT9j4
https://

2016_2 : 22
https://www.computer.org/csdl/journal/ca/2016/02/07265016/13rRUEgaruO
https://www.computer.org/csdl/journal/ca/2016/02/07797569/13rRUxNmPKF
https://www.computer.org/csdl/journal/ca/2016/02/07797570/13rRUxNmPFS
https://www.computer.org/csdl/journal/ca/2016/02/07163277/13rRUy0ZzUq
https://www.computer.org/csdl/journal/ca/2016/02/07299295/13rRUwwslyc
https://www.computer.org/csdl/journal/ca/2016/02/07265016/13rRUEgaruO
https://www.computer.org/csdl/journal/ca/2016/02/07327165/13rRUygT7A0
https://www.computer.org/csdl/journal/ca/2016/02/07155481/13rRUyekIZh
https://www.computer.org/csdl/journal/ca/2016/02/07222422/13rRUygT7zZ
https://www.computer.org/csdl/journal/ca/2016/02/07166306/13rRUyhaIiG
https://www.computer.org/csdl/journal/ca/2016/02/07159038/13rRUx0gebR
https://www.computer.org/csdl/journal/ca/2016/02/07307975/13rRUx0xPyw
https://www.computer.org/csdl/journal/ca/2016/02/07307120/13rRUEgarDj
https://www.computer.org/csdl/journal/ca/2016/02/07167661/13rRUIJcWnJ
https://

2013_2 : 15
https://www.computer.org/csdl/journal/ca/2013/02/lca20130200c1/13rRUwhpBSb
https://www.computer.org/csdl/journal/ca/2013/02/lca20130200c2/13rRUxjQyxc
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020037/13rRUyZaxsw
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020039/13rRUB6Sq4n
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020043/13rRUxlgy5H
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020047/13rRUxBa5h8
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020051/13rRUy08Myi
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020055/13rRUILLkIi
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020059/13rRUwgQptu
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020063/13rRUwcAqwG
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020067/13rRUEgs2v9
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020071/13rRUNvPLbI
https://www.computer.org/csdl/journal/ca/2013/02/lca2013020072/13rRUxly9fP
https://www.c

2006_1 : 11
https://www.computer.org/csdl/journal/ca/2006/01/y06001/13rRUxC0SQO
https://www.computer.org/csdl/journal/ca/2006/01/y06002/13rRUIJcWfz
https://www.computer.org/csdl/journal/ca/2006/01/y06003/13rRUyft7wG
https://www.computer.org/csdl/journal/ca/2006/01/y06004/13rRUxD9gZs
https://www.computer.org/csdl/journal/ca/2006/01/y06005/13rRUwciPhi
https://www.computer.org/csdl/journal/ca/2006/01/y06006/13rRUxjyX5Z
https://www.computer.org/csdl/journal/ca/2006/01/y06007/13rRUxOvebL
https://www.computer.org/csdl/journal/ca/2006/01/y06008/13rRUB6SpQE
https://www.computer.org/csdl/journal/ca/2006/01/y06009/13rRUxYINae
https://www.computer.org/csdl/journal/ca/2006/01/y06010/13rRUwInvLw
https://www.computer.org/csdl/journal/ca/2006/01/y06011/13rRUxBrGjG
2006_2 : 7
https://www.computer.org/csdl/journal/ca/2006/02/y06012/13rRUwwslw2
https://www.computer.org/csdl/journal/ca/2006/02/y06013/13rRUypp59N
https://www.computer.org/csdl/journal/ca/2006/02/y06014/13rRUIJuxxr
https://www.computer.org/

In [19]:
article_links[0]
len(article_links)

402

In [136]:
browser.get("https://www.computer.org/csdl/journal/ca/2018/01/07911285/13rRUwhpBHY")
element = WebDriverWait(browser, 10).until(
EC.presence_of_element_located((By.CLASS_NAME,'abstract-section-title'))
)
html = browser.page_source
soup = bs(html, 'html.parser')
journal = {}
journal['title'] = soup.select_one("div.article-max-width > csdl-periodical-article-metadata > div > div.article-title").text
journal['keyword'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > div.mt > span").text[9:]
journal['abstract'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-article-full-text > div.article-content.mt-lg").text[8:]
journal['date'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > span:nth-of-type(1)").text

In [137]:
print(journal['title'])
print(journal['keyword'])
print(journal['abstract'])
print(journal['date'])

Enabling Massive Multi-Threading with Fast Hashing
Instruction Sets, Computer Architecture, Hardware, Computational Modeling, Scheduling, Programming, Organizations, Dataflow, Hashing, Network On Chip, Thread Scheduling
The next generation of high-performance computers is expected to execute threads in orders of magnitude higher than today’s systems. Improper management of such huge amount of threads can create resource contention, leading to overall degraded system performance. By leveraging more practical approaches to distribute threads on the available resources, execution models and manycore chips are expected to overcome limitations of current systems. Here, we present DELTA—a Data-Enabled muLti-Threaded Architecture, where a producer-consumer scheme is used to execute threads via complete distributed thread management mechanism. We consider a manycore tiled-chip architecture where Network-on-Chip (NoC) routers are extended to support our execution model. The proposed extension i

In [20]:
from datetime import datetime

#파일명을 현재 시간으로
now = str(datetime.now())[0:19]
filename = ("./output/" + "%s" + "_output.txt") % now
filename = filename.replace(':','-')

start = time.time()

#크롤링 하는 함수
def crawl_abs(link) :
    browser.get(link)
    element = WebDriverWait(browser, 60).until(
    EC.presence_of_element_located((By.CLASS_NAME,'article-title'))
    )
    html = browser.page_source
    soup = bs(html, 'html.parser')
    
    journal = {}
    journal['title'] = soup.select_one("div.article-max-width > csdl-periodical-article-metadata > div > div.article-title").text
    journal['keyword'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > div.mt > span").text[9:]
    journal['abstract'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-article-full-text > div.article-content.mt-lg").text[8:]
    journal['date'] = soup.select_one("div.mb > csdl-periodical-article > div > div > div.col-md-6 > div > div.article-max-width > csdl-periodical-article-metadata > div > span:nth-of-type(1)").text
    
    with open(filename, 'a') as f:
        #각 요소를 tap으로 구분해서 쓰기
        o_data = '%s\t%s\t%s\t%s\n' % (journal['date'], journal['title'], journal['keyword'], journal['abstract'])
        #cp949 인코딩 문제 우선 임시적으로 이렇게 해결
        o_data = o_data.replace('\u2014','')
        o_data = o_data.replace('\u2013','')
        print(o_data)
        f.write(o_data)

for link in article_links :
    crawl_abs(link)
        
close = time.time()
print(close-start)

Jan.-June 2018, 	Partitioning Compute Units in CNN Acceleration for Statistical Memory Traffic Shaping	Bandwidth, Kernel, Acceleration, Convolution, Computer Architecture, Computational Modeling, Neural Networks, CNN, Traffic Shaping, Memory Bottleneck, Partitioning, Comput Units	Convolutional Neural Networks (CNNs) have become the default choice for processing visual information, and the design complexity of CNNs has been steadily increasing to improve accuracy. To cope with the massive amount of computation needed for such complex CNNs, the latest solutions utilize blocking of an image over the available dimensions (e.g., horizontal, vertical, channel, and kernel) and batching of multiple input images to improve data reuse in the memory hierarchy. While there has been a large collection of works on maximizing data reuse, only a few studies have focused on the memory bottleneck problem caused by limited bandwidth. Bandwidth bottleneck can easily occur in CNN acceleration as CNN layers

Jan.-June 2018, 	The EH Model: Analytical Exploration of Energy-Harvesting Architectures	Nonvolatile Memory, Mathematical Model, Computational Modeling, Analytical Models, Computer Architecture, Power System Reliability, Energy Harvesting, Intermittent Computing, Analytical Model	

Jan.-June 2018, 	SSD Performance Modeling Using Bottleneck Analysis	Resource Management, Predictive Models, Monitoring, Engines, Throughput, Feature Extraction, Interference	

Jan.-June 2018, 	MNCaRT: An Open-Source, Multi-Architecture Automata-Processing Research and Execution Ecosystem	Automata, Tools, Engines, Ecosystems, Field Programmable Gate Arrays, Benchmark Testing, Open Source Software, Automata, Accelerator Architectures, Software Tools, Open Source Software	

Jan.-June 2018, 	EZ-Pass: An Energy & Performance-Efficient Power-Gating Router Architecture for Scalable NoCs	Computer Architecture, Nickel, Routing, Latches, Ports Computers, Switches, Routing Protocols, Power Gating, Nework On Chips, Ener

AttributeError: 'NoneType' object has no attribute 'text'

In [139]:
#pandas를 이용하여 output파일을 읽어오기
data = pd.read_csv(filename, sep="\t", header=None)

#column 이름 설정
data.columns = ['Date','Title', 'Keyword', 'Abstracts']
data

,Date,Title,Keyword,Contents
0,"Jan.-June 2018,",Partitioning Compute Units in CNN Acceleration...,"Bandwidth, Kernel, Acceleration, Convolution, ...",Convolutional Neural Networks (CNNs) have beco...
1,"Jan.-June 2018,",Enabling Massive Multi-Threading with Fast Has...,"Instruction Sets, Computer Architecture, Hardw...",NaN
2,"Jan.-June 2018,",HMC-MAC: Processing-in Memory Architecture for...,"Random Access Memory, Memory Management, Regis...",NaN
3,"Jan.-June 2018,",Modeling Superscalar Processor Memory-Level Pa...,"Load Modeling, Analytical Models, Prefetching,...",NaN
4,"Jan.-June 2018,",Birkhoff-Von Neumann Switch Based on Greedy Sc...,"Switches, Random Access Memory, Ports Computer...",NaN
5,"Jan.-June 2018,",TLB Shootdown Mitigation for Low-Power Many-Co...,"Benchmark Testing, Servers, Hardware, Register...",Power efficiency has become one of the most im...
6,"Jan.-June 2018,",Accelerator for Sparse Machine Learning,"Sparse Matrices, Indexes, Random Access Memory...",Sparse matrix by vector multiplication (SpMV) ...
7,"Jan.-June 2018,",CF-TUNE: Collaborative Filtering Auto-Tuning f...,"Tuning, Collaboration, Microwave Integrated Ci...",NaN
8,"Jan.-June 2018,",Design of Generalized Pipeline Cellular Array ...,"Computer Architecture, Microprocessors, Pipeli...",NaN
9,"Jan.-June 2018,",CMA: A Reconfigurable Complex Matching Acceler...,"Computer Architecture, Coprocessors, Intrusion...",NaN


### /test

In [ ]:
print("Start Crawling volumes")
issueLinks = []
for i in range(0, len(journals)) :
    browser.get(url_main+journals[i])
    element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME,'nav-link'))
    )
    btn = browser.find_element_by_css_selector("a[aria-controls='pastIssuesMenu']")
    btn.click()

    year = 2018
    y_v = {}
    try:
        btn = browser.find_element_by_link_text("2010s")
        btn.click()
        while True :
            btn = browser.find_element_by_link_text( str(year) )
            btn.click()
            
            btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
            while i in range(0, len(btns)) :
                btns[i].click()
                element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME,'article-title'))
                )
                html = browser.page_source
                issueLinks = crawl_link(html, 'a.article-title')
            print(issueLinks)
            #y_v[year] = len(browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a"))
            #print(year, " ", len(browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")) )
            year -= 1
    except Exception:
        print("finish 2010s")
    
    print("lets go next")
    try:
        btn = browser.find_element_by_link_text("2000s")
        btn.click()
        while True :
            btn = browser.find_element_by_link_text( str(year) )
            btn.click()
            
            btns = browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")
            while i in range(0, len(btns)) :
                btns[i].click()
                element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME,'article-title'))
                )
                html = browser.page_source
                issueLinks = crawl_link(html, 'a.article-title')
            print(issueLinks)
            #y_v[year] = len(browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a"))
            #print(year, " ", len(browser.find_elements_by_css_selector("#pastIssuesMenu > div > div > div > div > a")) )
            year -= 1
    except Exception:
        print("finish 2000s")
    y_v
#Wait Until the page shown


Start Crawling volumes
finish 2010s
lets go next
finish 2000s
finish 2010s
lets go next
finish 2000s
['/csdl/journal/ca/2018/02/08263412/13rRUwwslBG', '/csdl/journal/ca/2018/02/08252923/13rRUzpzeDT', '/csdl/journal/ca/2018/02/08270368/13rRUxCitAA', '/csdl/journal/ca/2018/02/08246587/13rRUwciPho', '/csdl/journal/ca/2018/02/08289355/13rRUxAAT9E', '/csdl/journal/ca/2018/02/08307166/13rRUxly97z', '/csdl/journal/ca/2018/02/08328832/13rRUyft7wJ', '/csdl/journal/ca/2018/02/08331088/13rRUx0ge7W', '/csdl/journal/ca/2018/02/08331846/13rRUzpzeEL', '/csdl/journal/ca/2018/02/08326535/13rRUwbs1Ur', '/csdl/journal/ca/2018/02/08340153/13rRUILLkIj', '/csdl/journal/ca/2018/02/08341482/13rRUIIVleu', '/csdl/journal/ca/2018/02/08326533/13rRUwhpBSd', '/csdl/journal/ca/2018/02/08362750/13rRUwInv6s', '/csdl/journal/ca/2018/02/08329999/13rRUytWFbf', '/csdl/journal/ca/2018/02/08368305/13rRUxlgy5K', '/csdl/journal/ca/2018/02/08364629/13rRUxly97A', '/csdl/journal/ca/2018/02/08378258/13rRUy0ZzUs', '/csdl/journal/c

['/csdl/journal/ca/2018/02/08263412/13rRUwwslBG', '/csdl/journal/ca/2018/02/08252923/13rRUzpzeDT', '/csdl/journal/ca/2018/02/08270368/13rRUxCitAA', '/csdl/journal/ca/2018/02/08246587/13rRUwciPho', '/csdl/journal/ca/2018/02/08289355/13rRUxAAT9E', '/csdl/journal/ca/2018/02/08307166/13rRUxly97z', '/csdl/journal/ca/2018/02/08328832/13rRUyft7wJ', '/csdl/journal/ca/2018/02/08331088/13rRUx0ge7W', '/csdl/journal/ca/2018/02/08331846/13rRUzpzeEL', '/csdl/journal/ca/2018/02/08326535/13rRUwbs1Ur', '/csdl/journal/ca/2018/02/08340153/13rRUILLkIj', '/csdl/journal/ca/2018/02/08341482/13rRUIIVleu', '/csdl/journal/ca/2018/02/08326533/13rRUwhpBSd', '/csdl/journal/ca/2018/02/08362750/13rRUwInv6s', '/csdl/journal/ca/2018/02/08329999/13rRUytWFbf', '/csdl/journal/ca/2018/02/08368305/13rRUxlgy5K', '/csdl/journal/ca/2018/02/08364629/13rRUxly97A', '/csdl/journal/ca/2018/02/08378258/13rRUy0ZzUs', '/csdl/journal/ca/2018/02/08385180/13rRUxjyXcn', '/csdl/journal/ca/2018/02/08389213/13rRUxBJhxm', '/csdl/journal/ca/2

finish 2010s
lets go next
finish 2000s
['/csdl/journal/ca/2018/02/08263412/13rRUwwslBG', '/csdl/journal/ca/2018/02/08252923/13rRUzpzeDT', '/csdl/journal/ca/2018/02/08270368/13rRUxCitAA', '/csdl/journal/ca/2018/02/08246587/13rRUwciPho', '/csdl/journal/ca/2018/02/08289355/13rRUxAAT9E', '/csdl/journal/ca/2018/02/08307166/13rRUxly97z', '/csdl/journal/ca/2018/02/08328832/13rRUyft7wJ', '/csdl/journal/ca/2018/02/08331088/13rRUx0ge7W', '/csdl/journal/ca/2018/02/08331846/13rRUzpzeEL', '/csdl/journal/ca/2018/02/08326535/13rRUwbs1Ur', '/csdl/journal/ca/2018/02/08340153/13rRUILLkIj', '/csdl/journal/ca/2018/02/08341482/13rRUIIVleu', '/csdl/journal/ca/2018/02/08326533/13rRUwhpBSd', '/csdl/journal/ca/2018/02/08362750/13rRUwInv6s', '/csdl/journal/ca/2018/02/08329999/13rRUytWFbf', '/csdl/journal/ca/2018/02/08368305/13rRUxlgy5K', '/csdl/journal/ca/2018/02/08364629/13rRUxly97A', '/csdl/journal/ca/2018/02/08378258/13rRUy0ZzUs', '/csdl/journal/ca/2018/02/08385180/13rRUxjyXcn', '/csdl/journal/ca/2018/02/083

['/csdl/journal/ca/2018/02/08263412/13rRUwwslBG', '/csdl/journal/ca/2018/02/08252923/13rRUzpzeDT', '/csdl/journal/ca/2018/02/08270368/13rRUxCitAA', '/csdl/journal/ca/2018/02/08246587/13rRUwciPho', '/csdl/journal/ca/2018/02/08289355/13rRUxAAT9E', '/csdl/journal/ca/2018/02/08307166/13rRUxly97z', '/csdl/journal/ca/2018/02/08328832/13rRUyft7wJ', '/csdl/journal/ca/2018/02/08331088/13rRUx0ge7W', '/csdl/journal/ca/2018/02/08331846/13rRUzpzeEL', '/csdl/journal/ca/2018/02/08326535/13rRUwbs1Ur', '/csdl/journal/ca/2018/02/08340153/13rRUILLkIj', '/csdl/journal/ca/2018/02/08341482/13rRUIIVleu', '/csdl/journal/ca/2018/02/08326533/13rRUwhpBSd', '/csdl/journal/ca/2018/02/08362750/13rRUwInv6s', '/csdl/journal/ca/2018/02/08329999/13rRUytWFbf', '/csdl/journal/ca/2018/02/08368305/13rRUxlgy5K', '/csdl/journal/ca/2018/02/08364629/13rRUxly97A', '/csdl/journal/ca/2018/02/08378258/13rRUy0ZzUs', '/csdl/journal/ca/2018/02/08385180/13rRUxjyXcn', '/csdl/journal/ca/2018/02/08389213/13rRUxBJhxm', '/csdl/journal/ca/2